In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
import re
from drop_parse_file_id import parse_drop_file_id


from glitch.impedance import EISSpectrumDoc, SingleSpectrum

INPUT_FOLDER = "/Users/andreakowal/Downloads/"
OUTPUT_FOLDER = "/Users/andreakowal/Coding/Raw Nyquist Data" # where the raw Nyquist data points are being saved
OUTPUT_INTERCEPTS = "/Users/andreakowal/Coding/Intercept Data" # where the x-intercept values are being saved

# data folder path
data_folder = Path(INPUT_FOLDER + "DROP Experiments/FAS-50/NaCl/0.1 M").expanduser() 

# background correction files
OC_path = Path(INPUT_FOLDER + "MTRON/02_07_25_OCTest_C01.mpt").expanduser()
SC_path = Path(INPUT_FOLDER + "MTRON/02_03_25_SCTest_C01.mpt").expanduser()
oc = EISSpectrumDoc.from_eclab_mpt(OC_path)
sc = EISSpectrumDoc.from_eclab_mpt(SC_path)

# sort membrane files by membrane ID and replicate number
def extract_membrane_replicate_dropconc(file_path):
    membrane_ID = re.search(r"_(\d+)_", file_path.name)
    replicate = re.search(r"_R(\d+)_", file_path.name)
    drop_conc = re.search(r"_D([\d\.]+)M_", file_path.name)

    membrane_number = int(membrane_ID.group(1)) if membrane_ID else 999
    replicate_number = int(replicate.group(1)) if replicate else 999
    drop_conc_value = float(drop_conc.group(1)) if drop_conc else 999.0

    return (membrane_number, replicate_number, drop_conc_value)

nyquist_results = []

def trendline_intercept(Z_real, Z_imag):
    slope, intercept = np.polyfit(Z_real, Z_imag, 1)
    return -intercept / slope

for file_path in sorted(data_folder.glob("*.mpt"), key=extract_membrane_replicate_dropconc):
    print(f"Processing {file_path.name}")

    my_spectrum = EISSpectrumDoc.from_eclab_mpt(file_path)
    my_spectrum.background_correct(oc.cycles_raw[0], sc.cycles_raw[0])

    # filtering raw data
    raw_data = my_spectrum.cycles_raw[0]
    mask = raw_data.frequencies <= 250_000
    freqs_filtered = raw_data.frequencies[mask]
    impedance_filtered = raw_data.impedance[mask]

    Z_real = impedance_filtered.real
    Z_imag = impedance_filtered.imag

    df_points = pd.DataFrame({
        "Frequency (Hz)": freqs_filtered,
        "Z_real (Ohm)": Z_real,
        "Z_imag (Ohm)": Z_imag})

    # save raw Nyquist CVS files to output folder
    output_folder = Path(OUTPUT_FOLDER)
    output_filename = output_folder /f"{file_path.stem}_points.csv"
    df_points.to_csv(output_filename, index=False)
    print(f"Saved data table to: {output_filename}")

    # Parse ID for meta data
    id_string = file_path.stem
    print (id_string)
    meta_data = parse_drop_file_id(id_string)
    print (meta_data)

    # Creating cleaner title for graphs
    graph_title = f"{meta_data['Soak Concentration (M)']} {meta_data['Salt']}"

    # Build subtitles
    subtitle_parts = []

    if "Drop Concentration (M)" in meta_data:
        subtitle_parts.append(f"Drop Concentration (M): {meta_data['Drop Concentration (M)']}")

    subtitle_parts.append(f"Membrane: {meta_data['Membrane']}")
    subtitle_parts.append(f"ID: {meta_data['Membrane ID']}")
    subtitle_parts.append(f"Replicate: {meta_data['Replicate']}")
    subtitle_parts.append(f"Date: {meta_data["Date"]}")

    subtitle = " | ".join(subtitle_parts)
    
    combined_title = f"{graph_title}<br><span style='font-size:14px'>{subtitle}</span>"

    # Create trendline
    trendline = np.poly1d(np.polyfit(Z_real, Z_imag, 1))
    Z_imag_trend = trendline(Z_real)

    # Plotly Nyquist plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=Z_real,
        y=Z_imag, 
        mode='markers',
        name='EIS Data',
        marker=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=Z_real,
        y=Z_imag_trend,
        mode='lines',
        name='Trendline',
        line=dict(color='black')
    ))

    fig.update_layout(
        title= combined_title,
        xaxis_title='Re{Z} (Ohm)',
        yaxis_title='-Im{Z} (Ohm)',
        xaxis=dict(scaleanchor="y", scaleratio=1),  # equal aspect ratio
        yaxis=dict(),
        width=700,
        height=600
    )

    fig.show()

    filtered_cycle = SingleSpectrum(frequencies=freqs_filtered, impedance=impedance_filtered)
    filtered_doc = EISSpectrumDoc(cycles=[filtered_cycle])

    x_intercept = trendline_intercept(Z_real, Z_imag)

    nyquist_results.append({
        "Membrane ID": meta_data.get("Membrane ID"),
        "Replicate": meta_data.get("Replicate"),
        "Soak Concentration (M)": meta_data.get("Soak Concentration (M)"),
        "Drop Concentration (M)": meta_data.get("Drop Concentration (M)"),
        "Real intercept": x_intercept})

df = pd.DataFrame(nyquist_results)

df = pd.DataFrame(nyquist_results)
df["Drop Conc Numeric"] = df["Drop Concentration (M)"].str.replace(" M", "", regex=False).astype(float)

df_sorted = df.sort_values(by=["Membrane ID", "Replicate", "Drop Conc Numeric"], ascending=[True, True, True])

df_sorted = df_sorted.drop(columns=["Drop Conc Numeric"])

summary_file = f"/EIS_summary_{meta_data['Soak Concentration (M)'].replace(' ', '')}_{meta_data['Salt'].replace(' ', '_')}.csv"
df_sorted.to_csv(OUTPUT_INTERCEPTS + summary_file, index=False)
print(f"Saved summary file to: {OUTPUT_INTERCEPTS + summary_file}")
print(df_sorted)


Processing 0p1M_NaCl_D1M_01_R1_FAS50_20250606_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D1M_01_R1_FAS50_20250606_01_GEIS_C01_points.csv
0p1M_NaCl_D1M_01_R1_FAS50_20250606_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '01', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '1 M', 'Date': '06/06/2025'}


Processing 0p1M_NaCl_D2M_01_R1_FAS50_20250609_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D2M_01_R1_FAS50_20250609_01_GEIS_C01_points.csv
0p1M_NaCl_D2M_01_R1_FAS50_20250609_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '01', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '2 M', 'Date': '06/09/2025'}


Processing 0p1M_NaCl_D0p25M_01_R1_FAS50_20250603_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D0p25M_01_R1_FAS50_20250603_01_GEIS_C01_points.csv
0p1M_NaCl_D0p25M_01_R1_FAS50_20250603_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '01', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '0.25 M', 'Date': '06/03/2025'}


Processing 0p1M_NaCl_D0p5M_01_R1_FAS50_20250604_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D0p5M_01_R1_FAS50_20250604_01_GEIS_C01_points.csv
0p1M_NaCl_D0p5M_01_R1_FAS50_20250604_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '01', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '0.5 M', 'Date': '06/04/2025'}


Processing 0p1M_NaCl_D0p1M_01_R1_FAS50_20250602_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D0p1M_01_R1_FAS50_20250602_01_GEIS_C01_points.csv
0p1M_NaCl_D0p1M_01_R1_FAS50_20250602_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '01', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '0.1 M', 'Date': '06/02/2025'}


...

Processing 0p1M_NaCl_D2M_03_R1_FAS50_20250609_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D2M_03_R1_FAS50_20250609_01_GEIS_C01_points.csv
0p1M_NaCl_D2M_03_R1_FAS50_20250609_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '03', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '2 M', 'Date': '06/09/2025'}


Processing 0p1M_NaCl_D0p25M_03_R1_FAS50_20250603_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D0p25M_03_R1_FAS50_20250603_01_GEIS_C01_points.csv
0p1M_NaCl_D0p25M_03_R1_FAS50_20250603_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '03', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '0.25 M', 'Date': '06/03/2025'}


Processing 0p1M_NaCl_D0p5M_03_R1_FAS50_20250604_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D0p5M_03_R1_FAS50_20250604_01_GEIS_C01_points.csv
0p1M_NaCl_D0p5M_03_R1_FAS50_20250604_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '03', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '0.5 M', 'Date': '06/04/2025'}


Processing 0p1M_NaCl_D0p1M_03_R1_FAS50_20250602_01_GEIS_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/0p1M_NaCl_D0p1M_03_R1_FAS50_20250602_01_GEIS_C01_points.csv
0p1M_NaCl_D0p1M_03_R1_FAS50_20250602_01_GEIS_C01
{'Salt': 'NaCl', 'Membrane': 'FAS50', 'Membrane ID': '03', 'Replicate': '1', 'Soak Concentration (M)': '0.1 M', 'Drop Concentration (M)': '0.1 M', 'Date': '06/02/2025'}


Saved summary file to: /Users/andreakowal/Coding/Intercept Data/EIS_summary_0.1M_NaCl.csv
   Membrane ID Replicate Soak Concentration (M) Drop Concentration (M)  \
4           01         1                  0.1 M                  0.1 M   
2           01         1                  0.1 M                 0.25 M   
3           01         1                  0.1 M                  0.5 M   
0           01         1                  0.1 M                    1 M   
1           01         1                  0.1 M                    2 M   
8           02         1                  0.1 M                  0.1 M   
7           02         1                  0.1 M                 0.25 M   
9           02         1                  0.1 M                  0.5 M   
5           02         1                  0.1 M                    1 M   
6           02         1                  0.1 M                    2 M   
14          03         1                  0.1 M                  0.1 M   
12          03        